In [7]:
#!/usr/bin/env python3
import osmnx as ox
import geopandas as gpd
import folium
import pandas as pd

# --- Configuration ---
# Define a center point in NYC (example: Times Square)
center_point = (40.7580, -73.9855)  # (latitude, longitude)
radius = 300  # in meters

# --- Define OSM tags for each feature type ---
tags_steps = {'highway': 'steps'}
tags_elevator = {'highway': 'elevator'}
# For pavement surfaces, we query common walkable ways.
tags_surface = {'highway': ['footway', 'pedestrian']}

# --- Download Data ---
print("Downloading stairs data from OpenStreetMap...")
gdf_steps = ox.geometries_from_point(center_point, tags=tags_steps, dist=radius)
print(f"Found {len(gdf_steps)} stairs features.")

print("Downloading elevator data from OpenStreetMap...")
gdf_elevator = ox.geometries_from_point(center_point, tags=tags_elevator, dist=radius)
print(f"Found {len(gdf_elevator)} elevator features.")

print("Downloading pavement surface data from OpenStreetMap...")
gdf_surface = ox.geometries_from_point(center_point, tags=tags_surface, dist=radius)
print(f"Found {len(gdf_surface)} pavement surface features.")

# --- Optional: Compute Incline for Stairs ---
def compute_incline(geom):
    """
    Given a LineString geometry, compute the incline if the coordinates include elevation (z).
    Returns 'up' if the last point is higher than the first,
    'down' if lower, 'flat' if equal, or None if not computable.
    """
    try:
        if geom.geom_type == 'LineString':
            coords = list(geom.coords)
            # Check if coordinates include elevation data (i.e. a third value)
            if len(coords[0]) >= 3 and len(coords[-1]) >= 3:
                elevation_diff = coords[-1][2] - coords[0][2]
                if elevation_diff > 0:
                    return 'up'
                elif elevation_diff < 0:
                    return 'down'
                else:
                    return 'flat'
    except Exception:
        pass
    return None

# Add an 'incline' column to stairs data if not already present
if not gdf_steps.empty:
    if 'incline' not in gdf_steps.columns:
        gdf_steps['incline'] = None

    # For each stair feature, compute incline if possible
    for idx, row in gdf_steps.iterrows():
        if pd.isna(row['incline']):
            geom = row.geometry
            incline_val = None
            if geom.geom_type == 'LineString':
                incline_val = compute_incline(geom)
            elif geom.geom_type == 'MultiLineString':
                # For MultiLineString, compute incline for each part and decide by majority
                inclines = [compute_incline(line) for line in geom]
                inclines = [inc for inc in inclines if inc is not None]
                if inclines:
                    if inclines.count('up') > inclines.count('down'):
                        incline_val = 'up'
                    elif inclines.count('down') > inclines.count('up'):
                        incline_val = 'down'
                    else:
                        incline_val = inclines[0]
            if incline_val:
                gdf_steps.at[idx, 'incline'] = incline_val

# --- Create an Interactive Map with Folium ---
m = folium.Map(location=center_point, zoom_start=18)

def get_steps_color(incline):
    """Return a color based on the incline value for stairs."""
    if incline == 'up':
        return 'green'
    elif incline == 'down':
        return 'red'
    else:
        return 'blue'

# Plot stairs on the map
for idx, row in gdf_steps.iterrows():
    geom = row.geometry
    incline = row.get('incline', None)
    color = get_steps_color(incline)
    # folium expects coordinates in (lat, lon) order
    if geom.geom_type == 'LineString':
        coords = [(lat, lon) for lon, lat, *rest in geom.coords]
        folium.PolyLine(coords, color=color, weight=5,
                        tooltip=f"Stairs - Incline: {incline}").add_to(m)
    elif geom.geom_type == 'MultiLineString':
        for line in geom:
            coords = [(lat, lon) for lon, lat, *rest in line.coords]
            folium.PolyLine(coords, color=color, weight=5,
                            tooltip=f"Stairs - Incline: {incline}").add_to(m)

# Plot elevators on the map using a distinct color (purple)
for idx, row in gdf_elevator.iterrows():
    geom = row.geometry
    if geom.geom_type == 'Point':
        lat, lon = geom.y, geom.x
        folium.CircleMarker(location=(lat, lon), radius=5, color='purple',
                            fill=True, fill_color='purple',
                            tooltip="Elevator").add_to(m)
    elif geom.geom_type in ['LineString', 'MultiLineString']:
        if geom.geom_type == 'LineString':
            coords = [(lat, lon) for lon, lat, *rest in geom.coords]
            folium.PolyLine(coords, color='purple', weight=5,
                            tooltip="Elevator").add_to(m)
        elif geom.geom_type == 'MultiLineString':
            for line in geom:
                coords = [(lat, lon) for lon, lat, *rest in line.coords]
                folium.PolyLine(coords, color='purple', weight=5,
                                tooltip="Elevator").add_to(m)

# Plot pavement surfaces on the map (colored orange)
for idx, row in gdf_surface.iterrows():
    geom = row.geometry
    # Use the 'surface' key if available, otherwise show "unknown"
    surface_attr = row.get('surface', 'unknown')
    if geom.geom_type == 'LineString':
        coords = [(lat, lon) for lon, lat, *rest in geom.coords]
        folium.PolyLine(coords, color='orange', weight=5,
                        tooltip=f"Pavement Surface: {surface_attr}").add_to(m)
    elif geom.geom_type == 'MultiLineString':
        for line in geom:
            coords = [(lat, lon) for lon, lat, *rest in line.coords]
            folium.PolyLine(coords, color='orange', weight=5,
                            tooltip=f"Pavement Surface: {surface_attr}").add_to(m)
    elif geom.geom_type == 'Point':
        lat, lon = geom.y, geom.x
        folium.CircleMarker(location=(lat, lon), radius=3, color='orange',
                            fill=True, fill_color='orange',
                            tooltip=f"Pavement Surface: {surface_attr}").add_to(m)

# Save the map to an HTML file
m.save("map_with_stairs_elevators_surfaces.html")
print("Map saved as map_with_stairs_elevators_surfaces.html")


Found 33 stairs features.


/var/folders/8n/7kcjbmm92wv4f2ldlmlw82r40000gn/T/ipykernel_83421/2044859990.py:20: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release.
  gdf_steps = ox.geometries_from_point(center_point, tags=tags_steps, dist=radius)
/var/folders/8n/7kcjbmm92wv4f2ldlmlw82r40000gn/T/ipykernel_83421/2044859990.py:24: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release.
  gdf_elevator = ox.geometries_from_point(center_point, tags=tags_elevator, dist=radius)


Found 6 elevator features.


/var/folders/8n/7kcjbmm92wv4f2ldlmlw82r40000gn/T/ipykernel_83421/2044859990.py:28: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release.
  gdf_surface = ox.geometries_from_point(center_point, tags=tags_surface, dist=radius)


Found 292 pavement surface features.
Map saved as map_with_stairs_elevators_surfaces.html
